In [2]:
import pandas as pd
import plotly
import plotly.graph_objects as go

In [153]:
curves

,Curve,SQN,HGL,Discharge
0,9736_MH1 BasinCurve,1,0.000,0.0000
1,9736_MH1 BasinCurve,2,1.470,0.6610
2,9736_MH1 BasinCurve,3,2.334,1.4050
3,9736_MH1 BasinCurve,4,2.335,1.4065
4,9736_MH1 BasinCurve,5,4.590,5.0796
...,...,...,...,...
333,Welchlloyds-PumpCurve,2,4.500,0.0515
334,Welchlloyds-PumpCurve,3,6.000,0.0460
335,Welchlloyds-PumpCurve,4,11.000,0.0240
336,Welchlloyds-PumpCurve,5,16.000,0.0030


In [152]:
curves = pd.read_csv('PS_Curves.csv')
curves
df_ps

curve_ids = set()
for curve in list(curves.Curve.unique()):
    if curve in list(df_ps.Pump_Curve_Max) or curve in list(df_ps.Pump_Curve_Min):
        curve_ids.add(curve)
        
curve_ids
    

{'Welchlloyds-PumpCurve'}

In [149]:
curves.Curve.unique()

array(['9736_MH1 BasinCurve', 'Banbury-BasinCurve ', 'Banbury-PumpCurve',
       'Brichlynn-PumpCurve', 'Brichlynns-BasinCurve',
       'DegrittingChamber', 'GE1-BasinCurve ',
       'GE1-Pump1Curve-Extrapolated', 'GE1-PumpCurve1 ',
       'GE2-BasinCurve ', 'GE2-CompCurve1+2', 'GE2-PumpCurve1 ',
       'GE2-PumpCurve-Extrapolated', 'GE3-BasinCurve ',
       'GE3-CompCurve1+2', 'GE3-PumpCurve1', 'GE3-PumpCurve-Extrapolated',
       'GE4-BasinCurve ', 'GE4-CompCurve1+2', 'GE4-PumpCurve1',
       'GE5-BasinCurve ', 'GE5-CompCurve1+2', 'GE5-PumpCurve1 ',
       'GE5-PumpCurve1-10pct', 'GE5-PumpCurve11inch', 'Lynn-BasinCurve ',
       'Lynn-CompCurve1+2', 'Lynn-CompCurve3+4',
       'LynnNewSiphon200mnKnifeGateValve', 'Lynn-PumpCurve1 ',
       'Lynn-PumpCurve2', 'Lynn-PumpCurve3', 'Lynn-PumpCurve4',
       'Lynn-PumpCurve-Extrapolated', 'MH13B_McKay',
       'OldSyphonAbandonedRegulation', 'Rosallans-PumpCurve',
       'Rossallans-BasinCurve', 'Seashells-BasinCurve',
       'Seashells-Pum

In [159]:
df = pd.read_csv('PS_Info.csv')
curves = pd.read_csv('PS_Curves.csv')

pss = list(df.PS.unique())

f = open('PS_Summary.html', "w")
f.write('<link rel="stylesheet" href="style.css">\n')
f.write('<script src="script.js"></script>\n')
f.write('<link rel="stylesheet" href="style.css">\n')
f.write('<!DOCTYPE html>\n')
f.write('<html>\n')
f.write('<head>\n')
f.write('<meta charset="utf-8">\n')
f.write('</head>\n')
f.write('<body>\n\n')

f.write('<div class="tab">\n')
for i, tab in enumerate(pss):
    if i == -1: #Make tab active ----This gives wrong zoom levels so it is temporarily disabled until fixed
        f.write('  <button class="tablinks active" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
    else:
        f.write('  <button class="tablinks" onclick="openTab(event, ' + "'" + tab + "'"  + ')">' + tab + '</button>\n')
f.write('</div>\n')

for i, ps in enumerate(pss):
    
#     if ps == 'NW CSO Tank Cleanout Pumps':
    if 1 == 1:

        if i == -1: #Make tab active ----This gives wrong zoom levels so it is temporarily disabled until fixed
            f.write('<div id="' + ps + '" class="tabcontent" style="display:block">\n')
        else:
            f.write('<div id="' + ps + '" class="tabcontent">\n')    

        df_ps = df[df.PS==ps]

        fig = go.Figure()

        x = 1

        for sump in df_ps.Sump.unique():

            df_ps_sump = df_ps[df_ps.Sump==sump]
            invert_level = df_ps_sump.iloc[0,2]
            ground_level = df_ps_sump.iloc[0,3]

            fig.add_shape(type="rect",x0=x, y0=invert_level, x1=x+1, y1=ground_level,line=dict(color="RoyalBlue",width=1),)
            x_mid = x + 0.5

            fig.add_annotation(x=x_mid, y=ground_level + 0.5,
                text=sump,
                showarrow=False,
                arrowhead=1)

            x += 1.5

            y_axis_height = int(df_ps.Ground_Level.max())+2 - (int(df_ps.Invert_Level.min())-2)
            delta_pump_text_y = y_axis_height / 30
            pump_text_y_previous = -9999

            for index, row in df_ps_sump.iterrows():


                fig.add_shape(type="line",
                    x0=x, y0=row['Start_Level'], x1=x, y1=row['Stop_Level'],
                    line=dict(
                        color="Red",
                        width=2,
                    )
                )

                if row['Start_Level'] > ground_level:                                
#                     pump_text_y = ground_level + 0.5
                    
                    if ground_level + 0.5 - pump_text_y_previous < delta_pump_text_y:  #Avoid overlapping text                  
                        pump_text_y = pump_text_y_previous + delta_pump_text_y
                    else: 
                        pump_text_y = ground_level + 0.5
                    
                else:
                    if row['Start_Level'] - pump_text_y_previous < delta_pump_text_y:  #Avoid overlapping text                  
                        pump_text_y = pump_text_y_previous + delta_pump_text_y
                    else: 
                        pump_text_y = row['Start_Level'] + 0.5
                pump_text_y_previous = pump_text_y

                if ps == 'United PS':
                    print(pump_text_y_previous)

                fig.add_annotation(x=x, y=pump_text_y,
                    text=row['Pump'],
                    showarrow=False,
                    arrowhead=1)


                dot_line_ys = [row['Start_Level'],row['Stop_Level']]


                for dot_line_y in dot_line_ys:
                    fig.add_shape(type="line",
                        x0=x_mid, y0=dot_line_y, x1=x, y1=dot_line_y,
                        line=dict(
                            color="grey",
                            width=1,
                            dash="dot",
                        )
                    )  

                x += 0.5

            x = int(x) + 1

        fig['layout']['yaxis']['range']=[int(df_ps.Invert_Level.min())-2,int(df_ps.Ground_Level.max())+2]
        fig['layout']['xaxis']['range']=[0,x]
        
        fig.update_layout(
        autosize=False,
        width = 800,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=40,
            t=35,
            pad=4
            ),

        )

        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))  
        
        
        curve_ids = set()
        for curve in list(curves.Curve.unique()):            
            if curve in list(df_ps.Pump_Curve_Max) or curve in list(df_ps.Pump_Curve_Min):
                curve_ids.add(curve)
        
        if len(curve_ids) > 0:
            fig = go.Figure()
            for curve_id in curve_ids:
                df_curve = curves[curves.Curve==curve_id]
                fig.add_trace(go.Scatter(x=df_curve.Discharge, 
                             y = list(df_curve.HGL), 
                             mode='lines',name=curve_id))
                
            fig.update_layout(
            autosize=False,
            width = 800,
            height=200,
            margin=dict(
                l=50,
                r=50,
                b=40,
                t=35,
                pad=4
                ),

            )

            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))               
        
        df_pumps = df_ps[df_ps.columns[4:]].copy()        
   
#         f.write('<table class="first">\n')
        f.write('\n<table>\n')
        f.write('  <tr>\n')
        
        for column in df_pumps.columns:
            f.write('    <th>' + column + '</th>\n')
        
        f.write('  </tr>\n')

        for index, row in df_pumps.iterrows():
            f.write('  <tr>\n')
            for column in df_pumps.columns:
                if isinstance(row[column],str):
                    f.write('    <td>' + str(row[column]) + '</th>\n')
                else:
                    if np.isnan(row[column]):
                        f.write('    <td></th>\n')
                    else:
                        f.write('    <td>' + str(row[column]) + '</th>\n')
            f.write('  </tr>\n')                   
        f.write('</table>\n')
                        
  
        f.write('</div>\n')
        
f.write('</body>\n')
f.write('</html>\n')
f.close()




In [134]:
fig.show()

In [85]:
y_axis_height

12

In [86]:
delta_pump_text_y

0.6

In [87]:
pump_text_y_previous

-4.65

In [88]:
pump_text_y

-4.65

In [89]:
row['Start_Level']

-5.75

In [96]:
df_ps

,PS,Sump,Invert_Level,Ground_Level,Pump,Start_Level,Stop_Level,Speed,Setpoint,Capacity_Type,Discharge,Pump_Curve_Max,Pump_Curve_Min
110,White Rock PS,35445,-1.84,2.43,WhiteRock1a-1stLevel,0.31,-1.09,Constant,NaN,Q-DeltaH,NaN,WhiteRockPump-Up-to-date,NaN
111,White Rock PS,35445,-1.84,2.43,WhiteRock2a-2ndLevel,0.61,-0.94,Constant,NaN,Q-DeltaH,NaN,WhiteRockPump-Up-to-date,NaN
112,White Rock PS,35445,-1.84,2.43,WhiteRock3a-3rdLevel,0.91,-0.79,Constant,NaN,Q-DeltaH,NaN,WhiteRockPump-Up-to-date,NaN


In [105]:
df_pumps = df_ps[df_ps.columns[4:]].copy()
df_pumps

,Pump,Start_Level,Stop_Level,Speed,Setpoint,Capacity_Type,Discharge,Pump_Curve_Max,Pump_Curve_Min
110,WhiteRock1a-1stLevel,0.31,-1.09,Constant,NaN,Q-DeltaH,NaN,WhiteRockPump-Up-to-date,NaN
111,WhiteRock2a-2ndLevel,0.61,-0.94,Constant,NaN,Q-DeltaH,NaN,WhiteRockPump-Up-to-date,NaN
112,WhiteRock3a-3rdLevel,0.91,-0.79,Constant,NaN,Q-DeltaH,NaN,WhiteRockPump-Up-to-date,NaN


In [122]:
import numpy as np
np.isnan(df_pumps.loc[110,'Discharge'])

True